# Maximal Static Expansion for efficient parallelization on GPU

## Context

### Polly
My GSoC project is part of Polly. Polly is a loop and data-locality optimizer for LLVM. The optimisations are made using a mathematical model called polyhedral model. It models the memory access of the program. After modeling, transformations (tilling, loop fusion, loop unrolling ...) can be applied on the model to improve data-locality and/or parallelization. The aim of my project was to implement a transformation called Maximal Static Expansion (MSE).

### Maximal static expansion
Data-dependences in a program can lead to a very bad automatic parallelization. Modern compilers use techniques to reduce the number of such dependences. One of them is Maximal Static Expansion. The MSE is a transformation which expand the memory access to and from Array or Scalar. The goal is to disambiguate memory accesses by assigning different memory locations to non-conflicting writes. This method is described in a paper written by Denis Barthou, Albert Cohen and Jean-Francois Collard.[^f1] 
Let take a example (from the article) to understand the principle :

In [ ]:
int tmp;
for (int i = 0; i < N; i++) {
    tmp = i;
    for (int j = 0; j < N; j++) {
        tmp = tmp + i + j;
    }
    A[i] = tmp;
}

The data-dependences induced by tmp make the two loops unparallelizable : the iteration j of the inner-loop needs value from the previous iteration and it is impossible to parrallelize the i-loop because tmp is used in all iterations.

If we expand the accesses to tmp according to the outermost loop, we can then parallelize the i-loop.

In [ ]:
int tmp[N];
for (int i = 0; i < N; i++) {
    tmp[i] = i;
    for (int j = 0; j < N; j++) {
        tmp[i] = tmp[i] + i + j;
    }
    A[i] = tmp[i];
}

The accesses to tmp are now made to/from a different location for each iteration of the i-loop. It is then possible to execute the different iteration on different computation units (GPU, CPU ...).

### Static single assignement
Due to lack of time, I was not able to implement **maximal** static expansion but only fully-indexed expansion. The principle of fully-index expansion is that each write goes to a different memory location. 
Let see the idea on an example :

In [ ]:
int tmp;
for (int i = 0; i < N; i++) {
    tmp = i;
    for (int j = 0; j < N; j++) {
        B[j] = tmp + 3;
    }
    A[i] = B[i];
}

For the sake of simplicity, only the arrays will be expanded in this example. The fully expanded version is :

In [ ]:
int tmp;
for (int i = 0; i < N; i++) {
    tmp = i;
    for (int j = 0; j < N; j++) {
        B[i][j] = tmp + 3;
    }
    A[i] = B[i][i];
}

The details of fully-indexed expansion will be discussed in the following sections.


## My work
My project is part of Polly. I am a french student but during the GSoC I was a student at the university of Passau, Germany. The LooPo team welcome me and more especially Andreas Simbürger, one of my GSoC mentor and my master thesis supervisor. My other GSoC mentor is Michael Kruse, one of the main contributor to Polly, actually working in France.
I'd like to thank all the people that help and guide me and more especially Andreas and Michael. 

### JSON bug fix
### Allocate array on heap
### Array Fully indexed exp
### Scalar Fully indexed exp

## Evaluation

## Remaining work

### MAXIMAL expansion
### Select which SAI to expand

[^f1]: Denis Barthou, Albert Cohen, and Jean-François Collard. 2000. Maximal Static Expansion. Int. J. Parallel Program. 28, 3 (June 2000), 213-243. DOI=http://dx.doi.org/10.1023/A:1007500431910 
